In [ ]:
# 라이브러리 로드
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

In [ ]:
from google.colab import drive, files
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_excel('/content/drive/MyDrive/df_final_real.xlsx')

In [ ]:
# GPU가 사용 가능한지 확인하고 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# BERT 토크나이저 및 모델 불러오기
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=13)
model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [ ]:
# 레이블 매핑을 사건 종류에 따라 다르게 적용하는 함수 정의 (수정된 레이블 적용)
label_map = {
    '민사_승소': 0, '민사_패소': 1, '민사_기각': 2,
    '형사_기각': 3, '징역': 4, '벌금': 5, '무혐의': 6,
    '가사_승소': 7, '가사_패소': 8, '가사_기각': 9,
    '세무_승소': 10, '세무_패소': 11, '세무_기각': 12
}

In [ ]:
df = df[df['판결유형'].isin(label_map.keys())]

In [ ]:
# 텍스트 데이터 전처리 및 토큰화 함수 정의 (tqdm 추가)
def preprocess_and_tokenize(data, column_name):
    tokenized_data = []
    for idx, row in tqdm(data.iterrows(), total=len(data), desc="Tokenizing"):
        text = str(row[column_name])
        # 전처리: 불필요한 공백 제거
        text = " ".join(text.split())
        # 토큰화
        tokens = tokenizer.encode(text, add_special_tokens=True, max_length=512, truncation=True)
        tokenized_data.append(tokens)
    return tokenized_data

In [ ]:
# '판례내용' 열에 대해 토큰화 수행
df['판례내용_tokens'] = preprocess_and_tokenize(df, '판례내용')

Tokenizing: 100%|██████████| 42204/42204 [17:12<00:00, 40.87it/s]


In [ ]:
df.to_excel('df_real_token.xlsx')
files.download('df_real_token.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# 데이터셋 정의
class LegalDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        tokens = self.data.iloc[idx]['판례내용_tokens']
        tokens = torch.tensor(tokens, dtype=torch.long)
        label = torch.tensor(label_map[self.data.iloc[idx]['판결유형']], dtype=torch.long)
        return tokens, label

# Collate 함수 정의
def collate_fn(batch):
    tokens, labels = zip(*batch)
    tokens_padded = nn.utils.rnn.pad_sequence(tokens, batch_first=True, padding_value=tokenizer.pad_token_id)
    labels = torch.stack(labels)
    return tokens_padded, labels

# 데이터셋을 학습 및 검증 데이터로 분리 (7:3 비율)
train_size = int(0.7 * len(df))
valid_size = len(df) - train_size
train_data = df.iloc[:train_size]
valid_data = df.iloc[train_size:]

train_dataset = LegalDataset(train_data)
valid_dataset = LegalDataset(valid_data)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(valid_dataset, batch_size=8, shuffle=False, collate_fn=collate_fn)

In [ ]:
# 모델 및 토크나이저 설정
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=13)
model.to(device)

# Optimizer 및 가중치 설정
optimizer = AdamW(model.parameters(), lr=2e-5)

# 데이터 수에 따른 가중치 설정
class_counts = train_data['사건종류명'].value_counts().to_dict()
total_count = sum(class_counts.values())
class_weights = {case: total_count / count for case, count in class_counts.items()}
weights = torch.tensor([class_weights.get(case, 1.0) for case in train_data['사건종류명']], dtype=torch.float).to(device)
criterion = nn.CrossEntropyLoss(weight=weights)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# 학습 함수 정의
def train(model, loader, optimizer, epoch, log_interval=10):
    model.train()
    total_loss, total_correct = 0, 0
    total_batches = len(loader)
    all_labels = []
    all_preds = []

    progress_bar = tqdm(total=total_batches, desc=f"Training Epoch {epoch}", unit='batch')

    for batch_idx, (tokens, labels) in enumerate(loader):
        tokens, labels = tokens.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(tokens, labels=labels)
        loss, logits = outputs.loss, outputs.logits
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_correct += (logits.argmax(dim=1) == labels).sum().item()

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(logits.argmax(dim=1).cpu().numpy())

        if (batch_idx + 1) % (total_batches // 10) == 0:
            progress_bar.update(total_batches // 10)
            progress_bar.set_postfix(loss=total_loss / (batch_idx + 1), accuracy=total_correct / ((batch_idx + 1) * loader.batch_size))

    progress_bar.close()

    # 정밀도, 재현율, F1 스코어 계산
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')

    return total_loss / total_batches, total_correct / (total_batches * loader.batch_size), precision, recall, f1

# 검증 함수 정의
def validate(model, loader, epoch):
    model.eval()
    total_loss, total_correct = 0, 0
    total_batches = len(loader)
    all_labels = []
    all_preds = []

    progress_bar = tqdm(total=total_batches, desc=f"Validation Epoch {epoch}", unit='batch')

    with torch.no_grad():
        for batch_idx, (tokens, labels) in enumerate(loader):
            tokens, labels = tokens.to(device), labels.to(device)
            outputs = model(tokens, labels=labels)
            loss, logits = outputs.loss, outputs.logits
            total_loss += loss.item()
            total_correct += (logits.argmax(dim=1) == labels).sum().item()

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(logits.argmax(dim=1).cpu().numpy())

            if (batch_idx + 1) % (total_batches // 2) == 0:
                progress_bar.update(total_batches // 2)
                progress_bar.set_postfix(loss=total_loss / (batch_idx + 1), accuracy=total_correct / ((batch_idx + 1) * loader.batch_size))

    progress_bar.close()

    # 정밀도, 재현율, F1 스코어 계산
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')

    return total_loss / total_batches, total_correct / (total_batches * loader.batch_size), precision, recall, f1

In [ ]:
# 학습 루프 실행 및 조기 종료 설정
best_val_loss = float('inf')
early_stopping_patience = 3
early_stopping_counter = 0
min_epochs = 20  # 최소 에포크 설정, 어드바이스 반영 최대치로

train_accuracies, val_accuracies = [], []
train_losses, val_losses = [], []
precisions, recalls, f1_scores = [], [], []

with tqdm(range(20), desc="Training Epochs") as epochs:
    for epoch in epochs:  # 최대 20 에포크 설정
        train_loss, train_acc, train_prec, train_rec, train_f1 = train(model, train_loader, optimizer, epoch)
        val_loss, val_acc, val_prec, val_rec, val_f1 = validate(model, valid_loader, epoch)

        # 평가 지표 저장
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accuracies.append(train_acc)
        val_accuracies.append(val_acc)
        precisions.append(val_prec)
        recalls.append(val_rec)
        f1_scores.append(val_f1)

        # 조기 종료 조건 확인 및 모델 저장
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            early_stopping_counter = 0
            torch.save(model.state_dict(), 'model_weight_5.pth')  # 모델 저장
            print(f"Model saved at epoch {epoch}")
        else:
            early_stopping_counter += 1

        if early_stopping_counter >= early_stopping_patience and epoch >= min_epochs:
            print(f"Early stopping triggered at epoch {epoch}")
            break

# 평가 지표를 DataFrame으로 저장
metrics_df = pd.DataFrame({
    'train_accuracy': train_accuracies,
    'train_loss': train_losses,
    'val_accuracy': val_accuracies,
    'val_loss': val_losses,
    'precision': precisions,
    'recall': recalls,
    'f1_score': f1_scores
})

metrics_df.to_csv('training_metrics_full.csv', index=False)

Training Epoch 0:   0%|          | 0/3693 [00:00<?, ?batch/s]We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.

Training Epoch 0: 100%|█████████▉| 3690/3693 [10:17<00:00,  5.98batch/s, accuracy=0.887, loss=0.402]

Validation Epoch 0: 100%|█████████▉| 1582/1583 [01:21<00:00, 19.47batch/s, accuracy=0.94, loss=0.207]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
Training Epochs:   5%|▌         | 1/20 [11:39<3:41:33, 699.65s/it]

Model saved at epoch 0



Training Epoch 1: 100%|█████████▉| 3690/3693 [10:15<00:00,  6.00batch/s, accuracy=0.942, loss=0.187]

Validation Epoch 1: 100%|█████████▉| 1582/1583 [01:21<00:00, 19.50batch/s, accuracy=0.954, loss=0.148]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
Training Epochs:  10%|█         | 2/20 [23:17<3:29:36, 698.70s/it]

Model saved at epoch 1



Training Epoch 2: 100%|█████████▉| 3690/3693 [10:15<00:00,  5.99batch/s, accuracy=0.952, loss=0.152]

Validation Epoch 2: 100%|█████████▉| 1582/1583 [01:21<00:00, 19.47batch/s, accuracy=0.961, loss=0.146]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
Training Epochs:  15%|█▌        | 3/20 [34:56<3:17:57, 698.69s/it]

Model saved at epoch 2



Training Epoch 3: 100%|█████████▉| 3690/3693 [10:15<00:00,  5.99batch/s, accuracy=0.953, loss=0.143]

Validation Epoch 3: 100%|█████████▉| 1582/1583 [01:21<00:00, 19.47batch/s, accuracy=0.949, loss=0.185]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
Training Epoch 4: 100%|█████████▉| 3690/3693 [10:15<00:00,  5.99batch/s, accuracy=0.958, loss=0.13]

Validation Epoch 4: 100%|█████████▉| 1582/1583 [01:21<00:00, 19.48batch/s, accuracy=0.963, loss=0.131]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

Model saved at epoch 4



Training Epoch 5: 100%|█████████▉| 3690/3693 [10:15<00:00,  5.99batch/s, accuracy=0.96, loss=0.125]

Validation Epoch 5: 100%|█████████▉| 1582/1583 [01:21<00:00, 19.49batch/s, accuracy=0.943, loss=0.227]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
Training Epoch 6: 100%|█████████▉| 3690/3693 [10:15<00:00,  5.99batch/s, accuracy=0.962, loss=0.119]

Validation Epoch 6: 100%|█████████▉| 1582/1583 [01:21<00:00, 19.50batch/s, accuracy=0.938, loss=0.223]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

In [ ]:
files.download('model_weight_5.pth')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download('training_metrics_full.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **사건 유형별로 검증**

In [ ]:
# 사건 종류별로 데이터 분할
df_criminal = df[df['사건종류명'] == '형사']
df_civil = df[df['사건종류명'] == '민사']
df_family = df[df['사건종류명'] == '가사']
df_tax = df[df['사건종류명'] == '세무']

# 각 사건 유형별로 30%를 검증 데이터로 사용
valid_criminal = df_criminal.sample(frac=0.3, random_state=42)
valid_civil = df_civil.sample(frac=0.3, random_state=42)
valid_family = df_family.sample(frac=0.3, random_state=42)
valid_tax = df_tax.sample(frac=0.3, random_state=42)

# 검증용 데이터셋 및 데이터로더 정의
def create_dataloader(data):
    dataset = LegalDataset(data)
    return DataLoader(dataset, batch_size=8, shuffle=False, collate_fn=collate_fn)

valid_loader_criminal = create_dataloader(valid_criminal)
valid_loader_civil = create_dataloader(valid_civil)
valid_loader_family = create_dataloader(valid_family)
valid_loader_tax = create_dataloader(valid_tax)

In [ ]:
# 각 사건 유형별로 모델 검증
print("형사 사건 검증 결과:")
val_loss_criminal, val_acc_criminal, val_prec_criminal, val_rec_criminal, val_f1_criminal = validate(model, valid_loader_criminal, epoch=1)
print(f"Loss: {val_loss_criminal:.4f}, Accuracy: {val_acc_criminal:.4f}, Precision: {val_prec_criminal:.4f}, Recall: {val_rec_criminal:.4f}, F1-score: {val_f1_criminal:.4f}\n")

print("민사 사건 검증 결과:")
val_loss_civil, val_acc_civil, val_prec_civil, val_rec_civil, val_f1_civil = validate(model, valid_loader_civil, epoch=1)
print(f"Loss: {val_loss_civil:.4f}, Accuracy: {val_acc_civil:.4f}, Precision: {val_prec_civil:.4f}, Recall: {val_rec_civil:.4f}, F1-score: {val_f1_civil:.4f}\n")

print("가사 사건 검증 결과:")
val_loss_family, val_acc_family, val_prec_family, val_rec_family, val_f1_family = validate(model, valid_loader_family, epoch=1)
print(f"Loss: {val_loss_family:.4f}, Accuracy: {val_acc_family:.4f}, Precision: {val_prec_family:.4f}, Recall: {val_rec_family:.4f}, F1-score: {val_f1_family:.4f}\n")

print("세무 사건 검증 결과:")
val_loss_tax, val_acc_tax, val_prec_tax, val_rec_tax, val_f1_tax = validate(model, valid_loader_tax, epoch=1)
print(f"Loss: {val_loss_tax:.4f}, Accuracy: {val_acc_tax:.4f}, Precision: {val_prec_tax:.4f}, Recall: {val_rec_tax:.4f}, F1-score: {val_f1_tax:.4f}\n")

형사 사건 검증 결과:


Validation Epoch 1: 100%|██████████| 416/416 [00:21<00:00, 19.50batch/s, accuracy=0.977, loss=0.0892]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Loss: 0.0892, Accuracy: 0.9769, Precision: 0.7738, Recall: 0.7731, F1-score: 0.7734

민사 사건 검증 결과:


Validation Epoch 1: 100%|██████████| 950/950 [00:48<00:00, 19.48batch/s, accuracy=0.983, loss=0.0552]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Loss: 0.0552, Accuracy: 0.9834, Precision: 0.2779, Recall: 0.3483, F1-score: 0.2846

가사 사건 검증 결과:


Validation Epoch 1: 100%|██████████| 28/28 [00:01<00:00, 19.35batch/s, accuracy=0.754, loss=0.661]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Loss: 0.6610, Accuracy: 0.7545, Precision: 0.5012, Recall: 0.4552, F1-score: 0.4475

세무 사건 검증 결과:


Validation Epoch 1:  99%|█████████▉| 190/191 [00:09<00:00, 19.41batch/s, accuracy=0.943, loss=0.19]

Loss: 0.1889, Accuracy: 0.9411, Precision: 0.3433, Recall: 0.3186, F1-score: 0.3245




/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import pandas as pd
from google.colab import files

# 각 사건 유형별 검증 결과를 데이터프레임으로 저장
results = {
    '사건 유형': ['형사', '민사', '가사', '세무'],
    'Loss': [val_loss_criminal, val_loss_civil, val_loss_family, val_loss_tax],
    'Accuracy': [val_acc_criminal, val_acc_civil, val_acc_family, val_acc_tax],
    'Precision': [val_prec_criminal, val_prec_civil, val_prec_family, val_prec_tax],
    'Recall': [val_rec_criminal, val_rec_civil, val_rec_family, val_rec_tax],
    'F1-score': [val_f1_criminal, val_f1_civil, val_f1_family, val_f1_tax]
}

results_df = pd.DataFrame(results)

# CSV 파일로 저장
results_df.to_csv('validation_results.csv', index=False)

# CSV 파일 다운로드
files.download('validation_results.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import torch

# 모델 로드
model_path_2 = '/content/drive/MyDrive/model_4.pth'
model.load_state_dict(torch.load(model_path_2, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu')))
model.eval()  # 평가 모드 설정

In [ ]:
# 각 사건 유형별로 모델 검증
print("형사 사건 검증 결과:")
val_loss_criminal, val_acc_criminal, val_prec_criminal, val_rec_criminal, val_f1_criminal = validate(model, valid_loader_criminal, epoch=1)
print(f"Loss: {val_loss_criminal:.4f}, Accuracy: {val_acc_criminal:.4f}, Precision: {val_prec_criminal:.4f}, Recall: {val_rec_criminal:.4f}, F1-score: {val_f1_criminal:.4f}\n")

print("민사 사건 검증 결과:")
val_loss_civil, val_acc_civil, val_prec_civil, val_rec_civil, val_f1_civil = validate(model, valid_loader_civil, epoch=1)
print(f"Loss: {val_loss_civil:.4f}, Accuracy: {val_acc_civil:.4f}, Precision: {val_prec_civil:.4f}, Recall: {val_rec_civil:.4f}, F1-score: {val_f1_civil:.4f}\n")

print("가사 사건 검증 결과:")
val_loss_family, val_acc_family, val_prec_family, val_rec_family, val_f1_family = validate(model, valid_loader_family, epoch=1)
print(f"Loss: {val_loss_family:.4f}, Accuracy: {val_acc_family:.4f}, Precision: {val_prec_family:.4f}, Recall: {val_rec_family:.4f}, F1-score: {val_f1_family:.4f}\n")

print("세무 사건 검증 결과:")
val_loss_tax, val_acc_tax, val_prec_tax, val_rec_tax, val_f1_tax = validate(model, valid_loader_tax, epoch=1)
print(f"Loss: {val_loss_tax:.4f}, Accuracy: {val_acc_tax:.4f}, Precision: {val_prec_tax:.4f}, Recall: {val_rec_tax:.4f}, F1-score: {val_f1_tax:.4f}\n")

형사 사건 검증 결과:




Validation Epoch 1:   0%|          | 0/416 [00:00<?, ?batch/s]

In [ ]:
import pandas as pd
from google.colab import files

results = {
    '사건 유형': ['형사', '민사', '가사', '세무'],
    'Loss': [val_loss_criminal, val_loss_civil, val_loss_family, val_loss_tax],
    'Accuracy': [val_acc_criminal, val_acc_civil, val_acc_family, val_acc_tax],
    'Precision': [val_prec_criminal, val_prec_civil, val_prec_family, val_prec_tax],
    'Recall': [val_rec_criminal, val_rec_civil, val_rec_family, val_rec_tax],
    'F1-score': [val_f1_criminal, val_f1_civil, val_f1_family, val_f1_tax]
}

results_df = pd.DataFrame(results)

# 파일로 저장
results_df.to_csv('validation_results_2.csv', index=False)

# 파일 다운로드
files.download('validation_results_2.csv')